## PROGRAM extrct
Extracts data from the N-body output files Single.dat (fort.83) and Binary.dat (fort.82) relating to the overall cluster evolution, ie. half-mass radius and relaxation time, cluster mass, membership, core radius. 

Note: planets are ignored in this version. 

Output in extrct.dat: 

*   Number of single stars + binaries 
*   Number of binaries 
*   Time (Myr) 
*   Relaxation time (Myr) 
*   Total cluster mass (Msun) 
*   Mass in core (Msun) 
*   Mass outside the tidal radius (Msun) 
*   Maximum stellar distance from cluster centre of mass (pc) 
*   Half-mass radius (pc) 
*   Radius containing inner 10% of cluster mass (pc) 
*   Core radius - as determined by Nbody code (pc) 
*   Number of systems (stars + binaries) inside the half-mass radius   
*   Number of systems within 1pc of the cluster centre    
*   Number of systems within the inner lagrangian radius (10%)  
*   Number of systems within the core radius 
*   Velocity dispersion (km/s)

In [57]:
#Necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
import sys

## Reformatting and reading data from fort.83 and fort.82
(Removing format of fort.83 and fort.82 - run this just once)

In [4]:
import os.path

file83 = os.path.isfile('fort.83-awk')
file82 = os.path.isfile('fort.82-awk')

if not file83:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13}' fort.83 > fort.83-awk
    print "Removing format of fort.83"
if not file82:
    !awk '{print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12,$13,$14,$15,$16,$17,$18,$19,$20,$21,$22,$23,$24}' fort.82 > fort.82-awk
    print "Removing format of fort.82"

In [65]:
Ns = 0
Nb = 0
output = open('extrct.dat','r+')
with open("fort.83-awk", 'r') as file1:
    with open("fort.82-awk", 'r') as file2:
        while file1 and file2:
            line11 = file1.readline()
            line21 = file2.readline()
            if not (line11 and line21):
                break
            words11 = line11.split()
            words21 = line21.split()
            Ns = int(words11[0])
            Nb = int(words21[0])
            if Ns != -1000:
                line12 = file1.readline()
                words12 = line12.split()
                line13 = file1.readline()
                words13 = line13.split()
                blockS = np.fromfile(file1, sep=' ', count=13*Ns).reshape([Ns,13])
            if Nb != -1000:
                blockB = np.fromfile(file2, sep=' ', count=24*Nb).reshape([Nb,24])
            mtot = sum(blockS[:,2])+sum(blockB[:,8])+sum(blockB[:,9])
            data = str(Nsystems(blockS,blockB))+'\n'
            output.write(data)
output.close()
#for line in output:
#    line = line.strip()
    #columns = line.split()
    #name = columns[2]
    #j = float(columns[3])
    #print line

In [70]:
output = open('extrct.dat','r')
for line in output:
    #line = line.strip()
    column = line.split()
    print column[1]

167)
167)
163)
163)
163)
163)
163)
163)
162)
162)
162)
162)
162)
162)
162)
162)
162)
162)
162)
162)
162)
162)
160)
160)
160)
160)
160)
160)
160)
160)
159)
159)
159)
159)
157)
157)
157)
157)
156)
156)
156)
156)
155)
155)
154)
154)
154)
154)
151)
151)
150)
150)
149)
149)
146)
146)
145)
145)
145)
145)
144)
144)
142)
142)
142)
142)
142)
142)
142)
142)
141)
141)
140)
140)
140)
140)
140)
140)
137)
137)
137)
137)
136)
136)
136)
136)
135)
135)
132)
132)
130)
130)
127)
127)
127)
127)
126)
126)
125)
125)
122)
122)
120)
120)
119)
119)
119)
119)
118)
118)
118)
118)
116)
116)
115)
115)
115)
115)
115)
115)
115)
115)
115)
115)
115)
115)
114)
114)
114)
114)
113)
113)
113)
113)
113)
113)
112)
112)
112)
112)
111)
111)
111)
111)
111)
111)
111)
111)
111)
111)
111)
111)
111)
111)
111)
111)
110)
110)
110)
110)
110)
110)
110)
110)
110)
110)
109)
109)
109)
109)
109)
109)
109)
109)
109)
109)
109)
109)
109)
109)
109)
109)
108)
108)
108)
108)
108)
108)
108)
108)
108)
108)
106)
106)
105)
105)
105)
105)
105)
105)


In [7]:
#Numerical initiation of constants
amin = 1.0e+10
amax = -1.0e+10
vstar = -1
ebcut = 0
trhsum = 0
m0 = 0
rmax = 1
mtot = 0
mout = 0
n1 = 0
n2 = 0
n1pc = 0
vdisp = 0
msgl = 0
nsgl = 0
mwd = 0.0
nwd = 0

In [8]:
#Astronomical constants
pc = 3.0856776e+18
au = pc/1.4959787e+12

In [10]:
#Main call of functions and writing to output file

test=globaldata(words11,words12,words13,vstar,ebcut)

In [27]:
print test

(387.2089, 1.65, 2.96, 73204429.04385334, -1, 0)


In [9]:
#Global values
def globaldata(words11,words12,words13,vstar,ebcut):
    m0 = float(words13[0])
    rbar = float(words12[3])
    zmbar = float(words13[1])
    su = pc/(au*6.955e+10)*rbar*au
    if vstar == 0:
        vstar = 0.06557*np.sqrt(m0/rbar)
        print ' ZMBAR RBAR VSTAR ',m0,rbar,vstar
        ebcut = (1./zmbar)*(1./zmbar)/(20/su)
    return m0,rbar,zmbar,su,vstar,ebcut

In [ ]:
#Global structural parameters of the stellar cluster
def cstructure():
        rdS = np.sqrt((blockS[]-words[])**2+(blockS[]-words[])**2+(blockS[]-words[])**2)
        rdB = np.sqrt((blockB[]-words[])**2+(blockB[]-words[])**2+(blockB[]-words[])**2)

In [5]:
#Number of systems + number of binary systems
def Nsystems(blockS,blockB):
    nt = len(blockS)
    nb = len(blockB)
    return nt,nb

In [67]:
#OPEN(3,file='extrct.dat',status='unknown')